# 02 — Parsing & Cleaning
Objective: rename columns to canonical names, standardize units, parse goods description into model/capacity/price, and output clean processed CSV/XLSX.


In [ ]:
import pandas as pd
import re
from pathlib import Path
RAW = Path("data/raw/trade_data_2017_2025.xlsx")
PROC = Path("data/processed")
PROC.mkdir(parents=True, exist_ok=True)

df = pd.read_excel(RAW)
df.shape


In [ ]:
rename_map = {
    "HS CODE":"hsn_code",
    "GOODS DESCRIPTION":"goods_description",
    "Qty":"qty_raw",
    "QUANTITY":"quantity",
    "UNIT":"unit",
    "TOTAL VALUE_INR":"total_value_inr",
    "DUTY PAID_INR":"duty_paid_inr",
    "UNIT PRICE_USD":"unit_price_usd",
    "TOTAL VALUE_USD":"total_value_usd",
    "UNIT PRICE_INR":"unit_price_inr",
    "DATE":"date_raw",
    "PORT CODE":"port_code"
}
df = df.rename(columns=rename_map)
df.columns.tolist()


In [ ]:
def standardize_unit(u):
    if not isinstance(u, str): return None
    u2 = u.strip().lower()
    if u2 in ("pcs","pc","nos","pieces","piece","nos."):
        return "PCS"
    if u2 in ("kg","kgs","kilogram","kilograms"):
        return "KG"
    if u2 in ("mt","ton","tons","tonne"):
        return "MT"
    return u2.upper()

df['unit_standard'] = df.get('unit', '').apply(standardize_unit)
df['unit_standard'].value_counts().head(20)


In [ ]:
import numpy as np

def extract_usd_price(desc):
    if not isinstance(desc, str): return None
    # look for patterns like USD 1.5 or USD 1.50/PC or @ USD 2
    m = re.search(r'USD[\s:]*([0-9]+(?:\.[0-9]+)?)', desc.upper())
    if m: return float(m.group(1))
    # try $ symbol
    m2 = re.search(r'\$[\s]*([0-9]+(?:\.[0-9]+)?)', desc)
    if m2: return float(m2.group(1))
    return None

def extract_model_number(desc):
    if not isinstance(desc, str): return None
    m = re.search(r'([A-Z0-9]{2,}-\d{1,}|\b[A-Z]{2,}\d{2,}\b)', desc.upper())
    return m.group(1) if m else None

def extract_capacity(desc):
    if not isinstance(desc, str): return None
    m = re.search(r'(\d+(?:\.\d+)?\s*(?:ml|l|g|kg|cm|mm|inch|in))', desc.lower())
    return m.group(1) if m else None

df['unit_price_usd_extracted'] = df['goods_description'].apply(extract_usd_price)
df['model_number_extracted'] = df['goods_description'].apply(extract_model_number)
df['capacity_extracted'] = df['goods_description'].apply(extract_capacity)
df[['goods_description','unit_price_usd_extracted','model_number_extracted','capacity_extracted']].head(10)


In [ ]:
df['date_of_shipment'] = pd.to_datetime(df.get('date_raw'), dayfirst=True, errors='coerce')
df['year'] = df['date_of_shipment'].dt.year.astype('Int64')
df['month'] = df['date_of_shipment'].dt.month.astype('Int64')


In [ ]:
df['total_value_inr'] = pd.to_numeric(df.get('total_value_inr'), errors='coerce').fillna(0)
df['duty_paid_inr'] = pd.to_numeric(df.get('duty_paid_inr'), errors='coerce').fillna(0)
df['quantity'] = pd.to_numeric(df.get('quantity') or df.get('qty_raw'), errors='coerce').fillna(0)


In [ ]:
OUT = PROC / "trade_cleaned.csv"
df.to_csv(OUT, index=False)
print("Saved:", OUT)


notes: document assumptions (drop rows, fill 0s etc.)